<a href="https://colab.research.google.com/github/Gibbons-Lab/isb_course_2020/blob/master/micom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧫🦠 Modeling microbiota-wide metabolism with MICOM

This notebook will accompany the second session of the 2020 ISB Microbiome Course. The presentation slides can be [found here](https://gibbons-lab.github.io/isb_course_2020/micom). 

You can save your own local copy of this notebook by using `File > Save a copy in Drive`. You may be promted to cetify the notebook is safe. We promise that it is 🤞

**Disclaimer:**
The linear and quadratic programming problems MICOM has to solve are very large and very complicated. There are some very good commercial solvers that are very expensive (even though they are often free for academic use). To make this tutorial as accessible as possible we will use the Open Source solver [OSQP](https://osqp.org/). OSQP is amazing with quadratic programming problems (kudos!) but not as accurate for linear problems. Solvers usually only guarantee a solution within a certain numerical tolerance of the real solution. In order to make everything work with OSQP this tolerance has to be relaxed to about 10<sup>-3</sup>. This means that any result with an absolute value smaller than that might very well be zero so we should look at larger values only. Installing cost-free academic versions of commercial solvers like [IBM CPLEX](https://www.ibm.com/analytics/cplex-optimizer) or [Gurobi](https://www.gurobi.com/) would alow you to lower the tolerance to 10<sup>-6</sup>.



# 📝 Setup

MICOM installation is is usually pretty straight-forward and can be as easy as typing `pip install micom` into your Terminal. However, MICOM will require a solver for quadratic programming problems and all the best ones are commercial (boo) even though they often have free academic licenses 😌. We will use a brand new open source QP solver named OSQP, but this will require us to pull in development versions of certain packages.

But first let's start by downloading the materials again and switching to the folder.

In [ ]:
!git clone https://github.com/gibbons-lab/isb_course_2020 materials
%cd materials

## Basic Installation

Installing MICOM is straight-forward in Python. OSQP itself can be installed right along with it.

In [ ]:
!pip install -q osqp cobra micom

print("Done! 🎉 ")

## Enable QIIME 2 interactions

Finally we will need to install packages to read the "biom" format which is a file format QIIME 2 uses to save tables. This is only necessary if you want to read QIIME 2 FeatureTable artifacts.

In [ ]:
!pip install -q numpy Cython
!pip install -q biom-format

print("Done! 🎉 ")

Okay, all done. So let's get started 😁.

# 💻 MICOM

We will use the Python interface to MICOM since it plays nicely with Colaboratory. However, you could run the same steps within the QIIME 2 MICOM plugin ([q2-micom](https://library.qiime2.org/plugins/q2-micom/26/)). 

Here is an overview of all the steps and functions:
![micom overview](https://github.com/micom-dev/q2-micom/raw/706f583a060b91c12c0cec7acea2354fdd0dd320/docs/assets/overview.png).

MICOM starts from a combined abundance/taxonomy table, which MICOM abbreviates to a taxonomy table. To see how those tables look we can import MICOM and look at an example table:


In [ ]:
from micom.data import test_data

test_data().head()

The `file` column is not required when using a taxonomy database like we will do here. The `id` column specifies identifiers for the taxa and should be expressive and not include spaces or special characters. Each row needs to contain the the abundance of a single taxon in a single sample. 

Oh no, that's not what we have generated in the previous step. We only have separate QIIME 2 artifacts 😱 

No worries, we can deal with that.

## Importing data from QIIME 2

MICOM can read QIIME 2 artifacts. You don't even need to have QIIME 2 installed for that! But before we do so, let's resolve one issue. We discussed that MICOM summarizes genome-scale models into pangenome-scale models as a first step, but our data are on the ASV level...so how will we know what to summarize?

Basically, specific model database can be used to quickly summarize pangenome-scale models for use within MICOM. So, before we read our data we have to decide which model database to use. We will go with the [AGORA database](https://pubmed.ncbi.nlm.nih.gov/27893703/), which is a curated database of more than 800 bacterial strains that commonly live in the human gut. In particular, we will use a version of this database summarized on the genus rank which can be downloaded from [MICOM data repository](https://doi.org/10.5281/zenodo.3755182) which contains a whole lot of prebuilt databases.



In [ ]:
!wget -O agora103_genus.qza https://zenodo.org/record/3755182/files/agora103_genus.qza?download=1

Okay. We've got everything we need now. The data from the prior analysis can be found in the `treasure_chest` folder, so we can use those files. We will remove all taxa that make up less than 2.5% of the community to keep the models small and speed up this tutorial.

In [ ]:
from micom.taxonomy import qiime_to_micom

tax = qiime_to_micom(
    "treasure_chest/dada2/table.qza", 
    "treasure_chest/taxa.qza", 
    collapse_on="genus"
)

Notice the `collapse_on` argument. That will specify the rank on which to sumarize and can be a list of several ranks. When matching taxonomy you can either match by the particular rank of interest (for example, just comparing genus names here), or you could compare the entire taxonomy, which will require all taxonomic ranks prior to the target rank to match. For that you cloud specify `collapse_on=["kingdom", "phylum", "class", "order", "family", "genus"]`. 

Taxonomic names will often not match 100% between databases. For instance, the genus name "Prevotella" in one database may be "Prevotella_6" in another. The more ranks you use for matching the more likely are you to run into those issues. However, the more taxonomic ranks you use to match the more confident you can be that your observed taxon really is the same taxon as the one in the model database.

The resulting table will contain the same abundances but it will include more ranks if `collapse_on` is a list. All ranks present in the taxonomy will be used when matching to the database. The GreenGenes database is pretty old and many taxonomic names have been superceded by now. So we will stick with the "lax" option of only matching on genus ranks.

We can also look at the generated MICOM taxonomy.

In [ ]:
tax

One helpful thing to do is to merge in our metadata, so we'll have it at hand for the following steps.

In [ ]:
import pandas as pd

metadata = pd.read_table("metadata.tsv").rename(columns={"id": "sample_id"})
tax = pd.merge(tax, metadata, on="sample_id")
tax

With the taxonomic metadata, we can finally build our community-level models.

## Building community models

With the data we have now, building our models is pretty easy. We just pass our taxonomy table and model database to MICOM. We will have to specify where to write the models. We will also run that in parallel over two threads. It should take around 10 minutes to finish.

In [ ]:
from micom.workflows import build
from micom import Community
import pandas as pd

manifest = build(tax, "agora103_genus.qza", "models", solver="osqp", 
                 cutoff=2.5e-2, threads=2)


For different data a warning may pop up if less than 50% of the abundances can be matched to the database. If this happens, you can still continue, but be aware that such a sparse model may not accurately represent your sample. In lower-biomass 16S amplicon sequencing samples from stool, many reads can match to food components or to host mitochondria and these hits probably do not contribute much to bacterial community metabolism. These hits will be excluded from MICOM. 

We won't see any warnings here. So, we will go ahead for now. Let's also take a look what we got back from the `build` process.

In [ ]:
manifest

This will tell you many taxa were found in the database and what fraction of the total abundance was represented by the database. Looks okay here. 

So we now have our community models and can leverage MICOM fully by simulating community growth.

## Simulating growth

With our community models built, we can start to simulate growth with the cooperative tradeoff algorithm. Because we have no diet information for our samples, we will apply the same 'average Western Diet' to each individual. We will start by downloading this diet from the [MICOM data repository](https://doi.org/10.5281/zenodo.3755182).

In [ ]:
!wget -O western_diet_gut.qza https://zenodo.org/record/3755182/files/western_diet_gut.qza?download=1

This is again a QIIME 2 artifact, which we can load into MICOM.

In [ ]:
from micom.qiime_formats import load_qiime_medium

medium = load_qiime_medium("western_diet_gut.qza")
medium

Many dietary components get absorbed in the small intestine. This medium was created by taking dietary components and depleting all nutrients absorbed in the small intestine by a factor of 10 (indicated by the dilution column). 

Okay let's go right ahead and simulate growth. This will take a little while and give us time to dive into some details 🏊

In [ ]:
from micom.workflows import grow
import pickle

growth_results = grow(manifest, "models", medium, strategy="pFBA", tradeoff=0.5, threads=2)

# We'll save the results to a file
pickle.dump(growth_results, open("growth.pickle", "wb"))

If that takes too long or was aborted, we can read it in from the treasure chest.

In [ ]:
# Will only run if the previous step failed

import pickle

try:
  growth_results
except NameError:
  growth_results = pickle.load(open("treasure_chest/growth.pickle", "rb"))

What kind of results did we get? Well, `grow` returns a tuple of 3 data sets:

1. The predicted growth rate for all taxa in all samples
2. The import and export fluxes for each taxon and the external environment
3. Annotations for the fluxes mapping to other databases

The growth rates are pretty straightforward.

In [ ]:
growth_results.growth_rates.head()

More interesting are the exchange fluxes.

In [ ]:
growth_results.exchanges

So we see how much of each metabolite is either consumed or produced by each taxon in each sample. `tolerance` denotes the accuracy of the solver and tells you the smallest absolute flux that is likely different form zero (i.e. substantial flux). *All of the fluxes are normalized to 1g dry weight of bacteria*. So, you can directly compare fluxes between taxa, even if they are present at very different abundances. 

However, the metabolite names may not be very informative. That's why we have our annotations! For instance, to figure out what `ac[e]` is (air conditioning?), we can do the following:

In [ ]:
anns = growth_results.annotations
anns[anns.metabolite == "ac[e]"]

Ohhh, it's acetate. Yeah that makes more sense 🕵️‍♀️. For the AGORA models you can also use the official VMH knowledge base at https://vmh.life maintained by Dr. Thiele's, lab which will give you rich information on metabolites and reactions. For instance, you can find out a lot more about acetate at: https://www.vmh.life/#metabolite/ac. 

# 📊 Visualizations

Ok, we have seen that we generate a lot of output data from the growth simulations. But how do we make sense of it all? 

We will use the standard visualizations included in MICOM. These tools take in the growth results we obtained before and create visualizations in standalone HTML files that bundle the plots and raw data and can be viewed directly in your browser.

The first things we might want to look at are the growth rates for each taxon.

In [ ]:
from micom.viz import *

viz = plot_growth(growth_results)

Normally, we could call `viz.view()` afterwards and it would open it in our web browser. However, this will not work in Colab. However, the plot function create the file `growth_rates_[DATE].html` in your `materials` folder. To open it simply download that file and view it in your browser. We can see that there are many things going on, but it's not super clear. Let's continue.

## Growth niches

Two really important questions are 'what dietary nutrients are consumed by the microbiota and what metabolites do the microbiota produce?' We provided nutrients in our medium, but we don't actually know yet what was eaten by the microbiota. Let's check that out using the `plot_exchanges_per_sample` function.

In [ ]:
plot_exchanges_per_sample(growth_results)

We can have a look at the results after downloading `materials/sample_exchanges_[DATE].html`. It would be even better if we could visualize which taxa compete for similar resources. We can create a niche plot by using `plot_exchanges_per_taxon`.

In [ ]:
plot_exchanges_per_taxon(growth_results, perplexity=4, direction="import")


This function projects the full set of import or export fluxes onto a two dimensional plane, and arranges taxa so that more similar flux patterns lie close together. Thus, taxa closer to one another compete for a more similar set of resources (and/or produce a more similar set of metabolites). The center of the plot signifies a more competitive nutrient space, whereas clusters on the outskirts denote more isolated niches.

You can tune [TSNE parameters](https://distill.pub/2016/misread-tsne/), such as perplexity, to get a more meaningful grouping. We will lower the perplexity here since we don't have enough data.

## Metabolic connections to a phenotype

That is all nice, but how does that relate to recurrent *C. diff* infections? To answer that question we can use the `plot_fit` function. This will run a logistic regression with an L1 penalty on coefficients, with our disease status as the response variable and the normalizeed fluxes as independent variables. In general, import fluxes are not as predictive because, well, they are more relevant to the bacteria than us. What we usually care about (from a host-health perspective) are the production fluxes of metabolites. These are the total production fluxes into the extracellular (lumenal) space, which includes the set of metabolites that are bioavailable to the host. 

Because OSQP has a somewhat lower solver accuracy, we will be conservative for what we consider to be 'substantial flux' and will filter out fluxes smaller than 0.001 mmol/l.

In [ ]:
from micom.viz import *

manifest.index = manifest.sample_id
pheno = manifest.disease_stat

pl = plot_fit(growth_results, pheno, atol=1e-3, flux_type="production")


This will again create a file `fit_[DATE].html` that you can open. You will see the production fluxes most predictive of the phenotypes of interest and you can compare them across the groups. In the coefficient plot negative coefficients mean the particular metabolite is produced at a higher flux in the first group (recurrent C. diff in our case), whereas positive coefficients mean it is produced at a higher flux in the second group (healthy).

Some observations to help you intepret the results:

- riboflavin, or vitamin B, has [anti-inflammatory effects](https://academic.oup.com/ecco-jcc/article/14/5/595/5686400) in the human gut
- *C. diff.* toxins contain multipe aspartate residues and toxin B [has aspartate protease activity](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4885049/) 

Whatever we conclude here is based on *very* little data so it would be purely hypothetical. However, it may point us to future avenues to explore.

# 🏫 Exercises

Up to now, we have mostly used MICOM's "high-level" API, which is designed for working with several samples in parallel. However, MICOM also allows you to work with single models. We will choose a single sample now for further analysis.

First, let's recall what samples we had. 

In [ ]:
manifest

Let's look further into the surprising *C. diff.* individual that looked very similar to the healthy subjects. We will apply the same diet as before.

In [ ]:
from micom import load_pickle
from micom.qiime_formats import load_qiime_medium

medium = load_qiime_medium("western_diet_gut.qza")
medium.index = medium.reaction

com = load_pickle("models/ERR1883248.pickle")
com.medium = medium.flux
com

This is a MICOM community object. MICOM community models are full [COBRApy](https://opencobra.github.io/cobrapy/) models (with sprinkles on top) and there is a whole bunch of stuff we could do with it.

## Microbe-microbe interactions

Let's dive a bit more into competition and cooperation between taxa. We can start by simulating taxa knockouts using `com.knockout_taxa`. For that we will remove each of the taxa from the model (one-at-a-time) and see how that affects the growth rates of all other taxa. If other taxa grow faster after the knockout, they were competing with the knocked-out taxon. However, if they grow slower, they were cooperating with the knocked-out taxon. 

See the docs for more info: https://micom-dev.github.io/micom/taxa_knockouts.html.

Bonus points if you can visualize your results. How would you deal with vastly different scales of growth rates between taxa?

> Oh geez. I once saw somebody plot a heatmap with Seaborn using a Pandas DataFrame. I wish I could remember where that was... 🤔


In [ ]:
# Your code here

## The flux capacitator

Here is how you would run the cooperative tradeoff for a single model. We can follow that up with pFBA to get all fluxes in the system.

In [ ]:
sol = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
sol

The returned solution contains all fluxes in `sol.fluxes`. An `NaN` entry denotes that this reaction is not present in the organism.

In [ ]:
sol.fluxes.head()

The acetate export reaction has the name `EX_ac(e)`. Identify the primary acetate producer in the system. Don't forget about the accuracy of 10<sup>-3</sup>.

In [ ]:
# Your code here

Look up at least one other reaction at https://www.vmh.life/#microbes/reactions, and get it's predicted fluxes. Does the reaction take place? If yes in which organism? Can you identify the most active fluxes in the community?

In [ ]:
# Your code here

# 🔵 Addendum


## Choosing a tradeoff value

Even if you don't have growth rates available you can still use your data to choose a decent tradeoff value. This can be done by choosing the largest tradeoff value that still allows growth for the majority of the taxa that you observed in the sample (if they are present at an appreciable abundance, they should be able to grow). This can be done with the `tradeoff` workflow in MICOM that will run cooperative tradeoff with varying tradeoff values, which can be visualized with the `plot_tradeoff` function.

In [ ]:
from micom.workflows import tradeoff
import micom

tradeoff_results = tradeoff(manifest, "models", medium, threads=2)
tradeoff_results.to_csv("tradeoff.csv", index=False)

plot_tradeoff(tradeoff_results, tolerance=1e-4)

After opeing `tradeoff_[DATE].html` you will see that, for our example here, all tradeoff values work great. This is because we modeled very few taxa, which keeps the compettion down. If you would allow for fewer abundant taxa in the models, this would change drastically. For instance, here is an example from a colorectal cancer data set:

[![tradeoff example](https://micom-dev.github.io/micom/_images/tradeoff.png)](https://micom-dev.github.io/micom/_static/tradeoff.html)

You can see how not using the cooperative tradeoff would give you nonsense results where only 10% of all observed taxa grew. A tradeoff value of 0.6-0.8 would probably be a good choice for this particular data set.